# The Simpsons dataset

![](https://images.edrawmax.com/what-is/simpsons-family-tree/example.png) <br>
출처 : https://images.edrawmax.com/what-is/simpsons-family-tree/example.png

- 다운로드 : https://www.kaggle.com/datasets/pierremegret/dialogue-lines-of-the-simpsons?resource=download

# Quora dataset